In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

<br>

# Example 1: Simple network

A simple Bayesian network: 

$A \rightarrow B \rightarrow C$

In [3]:
edges = [
    ('C', 'B'), 
    ('B', 'A')
]

model = BayesianNetwork(edges)

<br>

A conditional probability tables:

In [7]:
p_ab = TabularCPD(
    variable = 'A', 
    variable_card = 2,
    values = [
        [.1, .7], 
        [.9, .3]
    ], 
    evidence = ['B'],
    evidence_card = [2]
)

p_bc = TabularCPD(
    variable = 'B', 
    variable_card = 2,
    values = [
        [.6, .5], 
        [.4, .5]
    ], 
    evidence = ['C'],
    evidence_card = [2]
)

p_c  = TabularCPD(
    variable = 'C', 
    variable_card = 2,    
    values = [
        [.2], [.8]
    ]
)

In [9]:
model.add_cpds(p_ab, p_bc, p_c)
model.check_model()

In [16]:
infer = VariableElimination(model)
res_0 = infer.query(["A"], evidence={"C" : 0})
res_1 = infer.query(["A"], evidence={"C" : 1})

In [17]:
print(res_0.variables)
print(res_0.values)

['A']
[0.34 0.66]


In [18]:
print(res_1.variables)
print(res_1.values)

['A']
[0.4 0.6]


<br>

# Example 2: Burglary network

A burglary network: 

$Burglary \rightarrow Alarm$; $Earthquake \rightarrow Alarm$; $Alarm \rightarrow MaryCalls$; $Alarm \rightarrow JohnCalls$

In [19]:
burglary_edges = [
    ("Burglary", "Alarm"),
    ("Earthquake", "Alarm"),
    ("Alarm", "JohnCalls"),
    ("Alarm", "MaryCalls"),
]
alarm_model = BayesianNetwork(burglary_edges)

<br>

A conditional probability tables:

In [20]:
cpd_burglary = TabularCPD(
    variable="Burglary", variable_card=2, values=[[0.999], [0.001]]
)
cpd_earthquake = TabularCPD(
    variable="Earthquake", variable_card=2, values=[[0.998], [0.002]]
)
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.999, 0.71, 0.06, 0.05], [0.001, 0.29, 0.94, 0.95]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)
cpd_johncalls = TabularCPD(
    variable="JohnCalls",
    variable_card=2,
    values=[[0.95, 0.1], [0.05, 0.9]],
    evidence=["Alarm"],
    evidence_card=[2],
)
cpd_marycalls = TabularCPD(
    variable="MaryCalls",
    variable_card=2,
    values=[[0.1, 0.7], [0.9, 0.3]],
    evidence=["Alarm"],
    evidence_card=[2],
)

In [23]:
alarm_model.add_cpds(
    cpd_burglary, 
    cpd_earthquake, 
    cpd_alarm, 
    cpd_johncalls, 
    cpd_marycalls
)
alarm_model.check_model()

True

In [26]:
infer = VariableElimination(alarm_model)
res_0 = infer.query(["MaryCalls"], evidence={"JohnCalls": 0})
res_1 = infer.query(["MaryCalls"], evidence={"JohnCalls": 1})

In [27]:
print(res_0.variables)
print(res_0.values)

['MaryCalls']
[0.10015929 0.89984071]


In [28]:
print(res_1.variables)
print(res_1.values)

['MaryCalls']
[0.12606263 0.87393737]
